In [69]:
import os
import glob
import warnings
import random
import json
import logging
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import zipfile
from typing import Tuple, List, Dict, Any, Optional
import copy
from pathlib import Path
from typing import List, Tuple
from collections import Counter, defaultdict

from PIL import Image
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
import torchvision.transforms as transforms
from torchvision.models import efficientnet_b7, EfficientNet_B7_Weights

from sklearn.model_selection import StratifiedKFold, train_test_split, GroupShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

import mlflow
import mlflow.pytorch
from mlflow.tracking import MlflowClient

warnings.filterwarnings('ignore')

In [52]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [53]:
class CNNLSTMConfig:
    # Data shape
    IMAGE_SIZE = (224, 224)     # resolusi tiap frame
    NUM_CLASSES = 2             # binary classification
    SEQ_LEN = 16                # jumlah frame per sequence

    # Model backbone
    MODEL_NAME = "efficientnet_b7"
    CNN_FEATURE_SIZE = 2560     # output EfficientNet-B7 (n_channels)
    LSTM_HIDDEN_SIZE = 512      # ukuran hidden state LSTM
    LSTM_NUM_LAYERS = 1
    DROPOUT = 0.4

    # Training params
    BATCH_SIZE = 2              # batch dalam sequence, jadi total frame = BATCH_SIZE*SEQ_LEN
    EPOCHS = 10
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 1e-4
    PATIENCE = 5
    MIN_DELTA = 0.001

    # Data augmentation
    ROTATION_RANGE = 15
    BRIGHTNESS_RANGE = 0.2
    CONTRAST_RANGE = 0.2

    # Directories
    DATA_DIR = "the_dataset"
    MODEL_DIR = "models/phase2_cnnlstm"
    RESULTS_DIR = "results/phase2_cnnlstm"
    LOGS_DIR = "logs"

    # MLflow
    MLFLOW_EXPERIMENT_NAME = "car_exit_detection_cnnlstm"
    MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

    def __init__(self):
        self.create_directories()
        self.setup_mlflow()

    def create_directories(self):
        directories = [self.MODEL_DIR, self.RESULTS_DIR, self.LOGS_DIR, "mlruns"]
        for dir_path in directories:
            os.makedirs(dir_path, exist_ok=True)
            logger.info(f"Created directory: {dir_path}")

    def setup_mlflow(self):
        mlflow.set_tracking_uri(self.MLFLOW_TRACKING_URI)
        try:
            mlflow.set_experiment(self.MLFLOW_EXPERIMENT_NAME)
            experiment = mlflow.get_experiment_by_name(self.MLFLOW_EXPERIMENT_NAME)
            if experiment is None:
                mlflow.create_experiment(self.MLFLOW_EXPERIMENT_NAME)
                logger.info(f"Created MLFlow experiment: {self.MLFLOW_EXPERIMENT_NAME}")
            else:
                logger.info(f"Using existing MLFlow experiment: {self.MLFLOW_EXPERIMENT_NAME}")
        except Exception as e:
            logger.error(f"MLFlow setup failed: {e}")
            raise


In [54]:
class CarExitSequenceDataset(Dataset):
    """
    Satu sample = 1 sequence berisi T frame (sampling/padding).
    Output: (x, y, length) di mana
      x: (T, C, H, W)  | y: int label  | length: banyak frame asli (<= T)
    """
    def __init__(self, sequences, transform, target_len=16):
        self.sequences = sequences
        self.transform = transform
        self.target_len = target_len

    def __len__(self): return len(self.sequences)

    def __getitem__(self, idx):
        item = self.sequences[idx]
        frames = item["frame_paths"]
        label = item["label"]

        # Sampling/padding agar panjang = target_len
        if len(frames) >= self.target_len:
            # uniform sampling sepanjang sequence
            sel_idx = np.linspace(0, len(frames)-1, self.target_len).astype(int)
            chosen = [frames[i] for i in sel_idx]
            length = self.target_len
        else:
            chosen = frames + [frames[-1]] * (self.target_len - len(frames))
            length = len(frames)

        imgs = []
        for fp in chosen:
            img = Image.open(fp).convert('RGB')
            imgs.append(self.transform(img))  # transform frame-level (resize/normalize/augment)

        x = torch.stack(imgs, dim=0)  # (T, C, H, W)
        return x, torch.tensor(label, dtype=torch.long), torch.tensor(length, dtype=torch.long)

def seq_collate(batch):
    xs, ys, ls = zip(*batch)
    return torch.stack(xs, 0), torch.stack(ys, 0), torch.stack(ls, 0)  # (B,T,C,H,W), (B,), (B,)

In [55]:
class CNNFeatureExtractor(nn.Module):
    """
    Ekstraktor fitur frame berbasis EfficientNet-B7 (tanpa classifier).
    - Jika input (B, C, H, W) -> output (B, D)
    - Jika input (B, T, C, H, W) -> output (B, T, D)
    D = cnn_feature_size (Eff-B7 = 2560).
    """
    def __init__(self, freeze_backbone: bool = True):
        super().__init__()
        self.backbone = efficientnet_b7(weights=EfficientNet_B7_Weights.IMAGENET1K_V1)
        self.backbone.classifier = nn.Identity()  # keluarkan vektor fitur akhir (B, 2560)

        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if x.dim() == 4:  # (B, C, H, W)
            return self.backbone(x)  # (B, D)

        elif x.dim() == 5:  # (B, T, C, H, W)
            B, T, C, H, W = x.shape
            x_flat = x.view(B * T, C, H, W)
            feats = self.backbone(x_flat)         # (B*T, D)
            D = feats.shape[1]
            return feats.view(B, T, D)            # (B, T, D)

        else:
            raise ValueError(f"Unexpected input shape: {x.shape}")

In [56]:
class SequenceHeadLSTM(nn.Module):
    """
    Head temporal LSTM untuk fitur per-frame.
    Input :
      feats   : (B, T, D)  -> fitur dari CNN per frame
      lengths : (B,)       -> panjang asli tiap sequence (<= T)
    Output:
      logits  : (B, num_classes)
    """
    def __init__(self, feat_dim: int = 2560, hidden: int = 512, layers: int = 1,
                 num_classes: int = 2, bidirectional: bool = False, dropout: float = 0.3):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=feat_dim,
            hidden_size=hidden,
            num_layers=layers,
            batch_first=True,
            bidirectional=bidirectional,
            dropout=(dropout if layers > 1 else 0.0),
        )
        out_dim = hidden * (2 if bidirectional else 1)
        self.classifier = nn.Sequential(
            nn.Linear(out_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes),
        )
        self._init_classifier()

    def _init_classifier(self):
        for m in self.classifier.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, feats: torch.Tensor, lengths: torch.Tensor) -> torch.Tensor:
        # feats: (B, T, D), lengths harus di CPU untuk pack
        packed = nn.utils.rnn.pack_padded_sequence(
            feats, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        _, (h_n, _) = self.lstm(packed)  # h_n: (num_layers * num_dirs, B, H)

        if self.lstm.bidirectional:
            h_last = torch.cat([h_n[-2], h_n[-1]], dim=1)  # (B, 2H)
        else:
            h_last = h_n[-1]                               # (B, H)

        logits = self.classifier(h_last)                   # (B, num_classes)
        return logits

In [57]:
class CNNLSTMModel(nn.Module):
    """
    Pembungkus: CNNFeatureExtractor + SequenceHeadLSTM.
    Input : x (B,T,C,H,W), lengths (B,)
    Output: logits (B,num_classes)
    """
    def __init__(self, num_classes=2, cnn_feature_size=2560,
                 lstm_hidden=512, lstm_layers=1, bidirectional=False, dropout=0.3,
                 freeze_backbone=True):
        super().__init__()
        self.extractor = CNNFeatureExtractor(freeze_backbone=freeze_backbone)
        self.head = SequenceHeadLSTM(
            feat_dim=cnn_feature_size,
            hidden=lstm_hidden,
            layers=lstm_layers,
            num_classes=num_classes,
            bidirectional=bidirectional,
            dropout=dropout
        )

    def forward(self, x_btchw: torch.Tensor, lengths: torch.Tensor) -> torch.Tensor:
        feats = self.extractor(x_btchw)            # (B, T, D)
        logits = self.head(feats, lengths)         # (B, num_classes)
        return logits


In [67]:
from time import asctime


class CNNLSTMTrainer:
    def __init__(self, config, device: torch.device):
        self.config = config
        self.device = device
        self.config.HEAD_LR = getattr(self.config, "HEAD_LR", 1e-3)
        self.config.SEQ_LEN = getattr(self.config, "SEQ_LEN", 16)

    def create_transforms(self) -> Tuple[transforms.Compose, transforms.Compose]:
        train_transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomResizedCrop(self.config.IMAGE_SIZE, scale=(0.85, 1.0)),
            transforms.RandomHorizontalFlip(p=0.3),
            transforms.RandomRotation(degrees=self.config.ROTATION_RANGE),
            transforms.ColorJitter(
                brightness=self.config.BRIGHTNESS_RANGE,
                contrast=self.config.CONTRAST_RANGE,
                saturation=0.1,
                hue=0.05
            ),
            transforms.RandomApply([transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.0))], p=0.1),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
            transforms.RandomErasing(p=0.1, scale=(0.02, 0.1)),
        ])

        val_transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.CenterCrop(self.config.IMAGE_SIZE),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
        ])
        return train_transform, val_transform

    def _build_head_optimizer(self, model: nn.Module) -> AdamW:
        head_params = list(model.head.parameters())
        optimizer = AdamW(head_params, lr=self.config.HEAD_LR, weight_decay=self.config.WEIGHT_DECAY)
        return optimizer

    def train_val_once(
        self,
        model: nn.Module,
        train_loader: DataLoader,
        val_loader: DataLoader,
        run_name: str = None
    ) -> Tuple[nn.Module, Dict, float]:
        
        is_active = mlflow.active_run() is not None

        with mlflow.start_run(run_name=run_name,nested = is_active):
            mlflow.log_params({
                'phase': 'Stage-1 (freeze CNN, train LSTM+FC)',
                'image_size': str(self.config.IMAGE_SIZE),
                'seq_len': self.config.SEQ_LEN,
                'batch_size': self.config.BATCH_SIZE,
                'epochs': self.config.EPOCHS,
                'head_lr': self.config.HEAD_LR,
                'weight_decay': self.config.WEIGHT_DECAY,
                'dropout': self.config.DROPOUT,
                'lstm_hidden': getattr(self.config, "LSTM_HIDDEN_SIZE", 512),
                'lstm_layers': getattr(self.config, "LSTM_NUM_LAYERS", 1),
                'backbone': getattr(self.config, "MODEL_NAME", "efficientnet_b7"),
                'freeze_backbone': True
            })

            criterion = nn.CrossEntropyLoss()
            optimizer = self._build_head_optimizer(model)
            scheduler = OneCycleLR(
                optimizer,
                max_lr=self.config.HEAD_LR,
                epochs=self.config.EPOCHS,
                steps_per_epoch=len(train_loader),
                pct_start=0.3
            )

            history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
            best_val_acc, best_state, patience_counter = 0.0, None, 0

            for epoch in range(self.config.EPOCHS):
                model.train()
                train_loss_sum, train_correct, train_total = 0.0, 0, 0
                pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{self.config.EPOCHS}', leave=False)

                for images, labels, lengths in pbar:
                    images = images.to(self.device)
                    labels = labels.to(self.device)
                    lengths = lengths.to(self.device)

                    optimizer.zero_grad()
                    outputs = model(images, lengths)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                    optimizer.step()
                    scheduler.step()

                    bs = labels.size(0)
                    train_loss_sum += loss.item() * bs
                    _, pred = outputs.max(1)
                    train_total += bs
                    train_correct += (pred == labels).sum().item()

                    pbar.set_postfix(loss=f'{loss.item():.4f}',
                                     acc=f'{100.0*train_correct/max(train_total,1):.1f}%')

                epoch_train_loss = train_loss_sum / max(train_total, 1)
                epoch_train_acc  = train_correct / max(train_total, 1)

                model.eval()
                val_loss_sum, val_correct, val_total = 0.0, 0, 0
                with torch.no_grad():
                    for images, labels, lengths in val_loader:
                        images = images.to(self.device)
                        labels = labels.to(self.device)
                        lengths = lengths.to(self.device)

                        outputs = model(images, lengths)
                        loss = criterion(outputs, labels)

                        bs = labels.size(0)
                        val_loss_sum += loss.item() * bs
                        _, pred = outputs.max(1)
                        val_total += bs
                        val_correct += (pred == labels).sum().item()

                epoch_val_loss = val_loss_sum / max(val_total, 1)
                epoch_val_acc  = val_correct / max(val_total, 1)

                history['train_loss'].append(epoch_train_loss)
                history['train_acc'].append(epoch_train_acc)
                history['val_loss'].append(epoch_val_loss)
                history['val_acc'].append(epoch_val_acc)

                mlflow.log_metrics({
                    'train_loss': epoch_train_loss,
                    'train_acc': epoch_train_acc,
                    'val_loss': epoch_val_loss,
                    'val_acc': epoch_val_acc
                }, step=epoch)

                logger.info(
                    f"Epoch {epoch+1}/{self.config.EPOCHS} "
                    f"Train: {epoch_train_loss:.4f}/{epoch_train_acc:.4f} "
                    f"Val: {epoch_val_loss:.4f}/{epoch_val_acc:.4f}"
                )

                if epoch_val_acc > best_val_acc + self.config.MIN_DELTA:
                    best_val_acc = epoch_val_acc
                    best_state = copy.deepcopy(model.state_dict())
                    patience_counter = 0
                    logger.info(f"New best model — Val Acc: {epoch_val_acc:.4f}")
                else:
                    patience_counter += 1
                    if patience_counter >= self.config.PATIENCE:
                        logger.info(f"Early stopping at epoch {epoch+1}")
                        break

            if best_state is not None:
                model.load_state_dict(best_state)

            os.makedirs(self.config.MODEL_DIR, exist_ok=True)
            best_model_path = os.path.join(self.config.MODEL_DIR, 'best_cnnlstm_stage1.pth')
            torch.save({'model_state_dict': model.state_dict(),
                        'best_val_acc': best_val_acc,
                        'config': self.config.__dict__}, best_model_path)
            mlflow.log_artifact(best_model_path, "models")

            return model, history, best_val_acc

In [59]:
def load_sequence_index(data_dir: str) -> List[Dict]:
    """
    Return list of sequence dict:
      [
        {"sequence_id": "0_seq001", "frame_paths": [...], "label": 0},
        {"sequence_id": "1_seq123", "frame_paths": [...], "label": 1},
        ...
      ]
    Struktur diasumsikan: data_dir/{0,1}/{seq_name}/*.jpg|png|jpeg|bmp
    """
    root = Path(data_dir)
    exts = {'.jpg', '.jpeg', '.png', '.bmp'}
    sequences: List[Dict] = []

    for cls in [0, 1]:
        class_dir = root / str(cls)
        if not class_dir.exists():
            logger.warning(f"Class directory {class_dir} not found")
            continue

        for seq_dir in sorted([p for p in class_dir.iterdir() if p.is_dir()]):
            frames = sorted([f for f in seq_dir.iterdir()
                             if f.is_file() and f.suffix.lower() in exts])
            if not frames:
                logger.warning(f"No images in sequence {seq_dir}")
                continue

            sequences.append({
                "sequence_id": f"{cls}_{seq_dir.name}",
                "frame_paths": [str(x) for x in frames],
                "label": int(cls),
            })

    if not sequences:
        raise ValueError("No sequences found.")

    logger.info(f"Loaded {len(sequences)} sequences "
                f"({sum(len(s['frame_paths']) for s in sequences)} frames total)")
    return sequences

def split_train_val_sequences(sequences, val_size=0.2, seed=42):
    """
    Split sekali anti-leak: semua frame dari 1 sequence jatuh ke train ATAU val saja.
    """
    groups = [s["sequence_id"] for s in sequences]
    labels = [s["label"] for s in sequences]

    gss = GroupShuffleSplit(n_splits=1, test_size=val_size, random_state=seed)
    train_idx, val_idx = next(gss.split(sequences, labels, groups))

    train_seq = [sequences[i] for i in train_idx]
    val_seq   = [sequences[i] for i in val_idx]

    logger.info(f"Train sequences: {len(train_seq)} | Val sequences: {len(val_seq)}")
    return train_seq, val_seq


In [60]:
def visualize_trainval_history(history: Dict, config, title: str = "CNN+LSTM Stage-1 (Freeze CNN)",
                               best_val_acc: float = None, filename: str = "trainval_history_stage1.png"):
    """
    history: dict dengan keys:
        - 'train_acc', 'val_acc', 'train_loss', 'val_loss'  (list per-epoch)
    config : punya RESULTS_DIR untuk lokasi simpan
    """
    train_acc = np.array(history.get('train_acc', []), dtype=float)
    val_acc   = np.array(history.get('val_acc', []), dtype=float)
    train_ls  = np.array(history.get('train_loss', []), dtype=float)
    val_ls    = np.array(history.get('val_loss', []), dtype=float)

    epochs = np.arange(1, len(train_acc) + 1)

    plt.figure(figsize=(12, 5))

    # Acc panel
    ax1 = plt.subplot(1, 2, 1)
    ax1.plot(epochs, train_acc, label='Train Acc', linewidth=2)
    ax1.plot(epochs, val_acc,   label='Val Acc',   linewidth=2)
    ax1.set_xlabel('Epoch'); ax1.set_ylabel('Accuracy')
    ax1.set_title('Accuracy per Epoch')
    ax1.grid(True, alpha=0.3); ax1.legend()

    # Loss panel
    ax2 = plt.subplot(1, 2, 2)
    ax2.plot(epochs, train_ls, label='Train Loss', linewidth=2)
    ax2.plot(epochs, val_ls,   label='Val Loss',   linewidth=2)
    ax2.set_xlabel('Epoch'); ax2.set_ylabel('Loss')
    ax2.set_title('Loss per Epoch')
    ax2.grid(True, alpha=0.3); ax2.legend()

    suptitle = title if best_val_acc is None else f"{title}  |  Best Val Acc: {best_val_acc:.4f}"
    plt.suptitle(suptitle, fontsize=14, fontweight='bold')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    os.makedirs(config.RESULTS_DIR, exist_ok=True)
    out_path = os.path.join(config.RESULTS_DIR, filename)
    plt.savefig(out_path, dpi=300, bbox_inches='tight')
    plt.show()

    mlflow.log_artifact(out_path)
    return out_path

In [61]:
def evaluate_test_set_sequence(
    model: torch.nn.Module,
    test_loader: torch.utils.data.DataLoader,
    device: torch.device,
    config
) -> dict:
    model.eval()
    test_correct, test_total = 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels, lengths in tqdm(test_loader, desc="Evaluating test set"):
            images  = images.to(device)    # (B,T,C,H,W)
            labels  = labels.to(device)    # (B,)
            lengths = lengths.to(device)   # (B,)

            outputs = model(images, lengths)          # (B,num_classes)
            _, pred = torch.max(outputs.data, dim=1)  # (B,)

            test_total   += labels.size(0)
            test_correct += (pred == labels).sum().item()

            all_preds.extend(pred.cpu().numpy().tolist())
            all_labels.extend(labels.cpu().numpy().tolist())

    # -- Metrics (dengan zero_division=0 agar aman kalau ada kelas kosong)
    accuracy   = test_correct / max(test_total, 1)
    f1_w       = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    prec_w     = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    rec_w      = recall_score(all_labels, all_preds, average='weighted', zero_division=0)

    f1_per     = f1_score(all_labels, all_preds, average=None, zero_division=0)
    prec_per   = precision_score(all_labels, all_preds, average=None, zero_division=0)
    rec_per    = recall_score(all_labels, all_preds, average=None, zero_division=0)

    metrics = {
        'accuracy': accuracy,
        'f1_weighted': f1_w,
        'precision_weighted': prec_w,
        'recall_weighted': rec_w,
        'f1_normal': f1_per[0] if len(f1_per) > 0 else 0.0,
        'f1_person_exiting': f1_per[1] if len(f1_per) > 1 else 0.0,
        'precision_normal': prec_per[0] if len(prec_per) > 0 else 0.0,
        'precision_person_exiting': prec_per[1] if len(prec_per) > 1 else 0.0,
        'recall_normal': rec_per[0] if len(rec_per) > 0 else 0.0,
        'recall_person_exiting': rec_per[1] if len(rec_per) > 1 else 0.0,
        'test_samples': test_total
    }

    # -- Confusion Matrix
    class_names = ['Normal Parking', 'Person Exiting']
    cm = confusion_matrix(all_labels, all_preds, labels=[0,1])

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Test Set Confusion Matrix', fontweight='bold', fontsize=14)
    plt.ylabel('True Label'); plt.xlabel('Predicted Label')

    os.makedirs(config.RESULTS_DIR, exist_ok=True)
    cm_path = os.path.join(config.RESULTS_DIR, 'test_confusion_matrix_seq.png')
    plt.savefig(cm_path, dpi=300, bbox_inches='tight'); plt.show()
    mlflow.log_artifact(cm_path)

    # -- Classification report (log ke logger)
    report = classification_report(all_labels, all_preds, target_names=class_names, zero_division=0)
    logger.info("\nTEST SET EVALUATION (SEQUENCE):")
    logger.info(f"Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    logger.info(f"F1 (weighted): {f1_w:.4f} | Precision (weighted): {prec_w:.4f} | Recall (weighted): {rec_w:.4f}")
    logger.info(f"\nDetailed Classification Report:\n{report}")

    return metrics

In [62]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def main():
    try:
        # ---------- setup ----------
        set_seed(42)
        config = CNNLSTMConfig()  # ganti ke config sequence kamu (bukan CNNConfig lama)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"Device: {device}")
        if torch.cuda.is_available():
            mem_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
            logger.info(f"GPU Memory: {mem_gb:.1f} GB")

        # ---------- load sequences ----------
        sequences = load_sequence_index(config.DATA_DIR)  # -> list of dicts (sequence-level)
        class_counts = {
            0: sum(1 for s in sequences if s["label"] == 0),
            1: sum(1 for s in sequences if s["label"] == 1),
        }
        logger.info(f"Sequence count: {len(sequences)} | Class dist (seq): {class_counts}")

        # sekali split train/val tanpa test (anti-leak per sequence)
        train_seq, val_seq = split_train_val_sequences(sequences, val_size=0.2, seed=42)
        logger.info(f"Train sequences: {len(train_seq)} | Val sequences: {len(val_seq)}")

        # ---------- transforms ----------
        trainer = CNNLSTMTrainer(config, device)
        train_tf, val_tf = trainer.create_transforms()

        # ---------- datasets & loaders ----------
        train_ds = CarExitSequenceDataset(train_seq, transform=train_tf, target_len=config.SEQ_LEN)
        val_ds   = CarExitSequenceDataset(val_seq,   transform=val_tf,   target_len=config.SEQ_LEN)

        train_loader = DataLoader(
            train_ds,
            batch_size=config.BATCH_SIZE,
            shuffle=True,
            num_workers=0,
            pin_memory=(device.type == 'cuda'),
            collate_fn=seq_collate
        )
        val_loader = DataLoader(
            val_ds,
            batch_size=config.BATCH_SIZE,
            shuffle=False,
            num_workers=0,
            pin_memory=(device.type == 'cuda'),
            collate_fn=seq_collate
        )

        # ---------- model (Stage-1: freeze backbone) ----------
        model = CNNLSTMModel(
            num_classes=config.NUM_CLASSES,
            cnn_feature_size=config.CNN_FEATURE_SIZE,   # 2560 for EffNet-B7
            lstm_hidden=config.LSTM_HIDDEN_SIZE,
            lstm_layers=config.LSTM_NUM_LAYERS,
            bidirectional=False,
            dropout=config.DROPOUT,
            freeze_backbone=True,                       # penting: bekukan CNN
        ).to(device)

        # sanity log: jumlah param trainable vs frozen
        trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
        frozen    = sum(p.numel() for p in model.parameters() if not p.requires_grad)
        logger.info(f"Trainable params: {trainable:,} | Frozen params: {frozen:,}")

        # ---------- train once (no K-Fold) ----------
        run_name = f"cnn_lstm_stage1_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        best_model, history, best_val_acc = trainer.train_val_once(model, train_loader, val_loader, run_name=run_name)

        # ---------- visualize ----------
        plot_path = visualize_trainval_history(
            history, config,
            title="CNN+LSTM Stage-1 (Freeze CNN)",
            best_val_acc=best_val_acc,
            filename="trainval_history_stage1.png"
        )
        logger.info(f"Saved plot: {plot_path}")

        # ---------- save summary ----------
        results_summary = {
            'timestamp': datetime.now().isoformat(),
            'phase': 'Stage 1 - CNN+LSTM (freeze CNN, train LSTM+FC)',
            'model_architecture': 'EfficientNet-B7 (frozen) + LSTM + FC',
            'config': {
                'model_name': config.MODEL_NAME,
                'image_size': config.IMAGE_SIZE,
                'seq_len': config.SEQ_LEN,
                'batch_size': config.BATCH_SIZE,
                'epochs': config.EPOCHS,
                'head_lr': getattr(config, "HEAD_LR", 1e-3),
                'weight_decay': config.WEIGHT_DECAY,
                'dropout': config.DROPOUT,
                'lstm_hidden': config.LSTM_HIDDEN_SIZE,
                'lstm_layers': config.LSTM_NUM_LAYERS,
                'freeze_backbone': True
            },
            'data_info': {
                'total_sequences': len(sequences),
                'train_sequences': len(train_seq),
                'val_sequences': len(val_seq),
                'class_distribution_sequences': class_counts
            },
            'final_val': {
                'best_val_acc': best_val_acc,
                'epochs_ran': len(history.get('val_acc', []))
            }
        }

        os.makedirs(config.RESULTS_DIR, exist_ok=True)
        results_path = os.path.join(config.RESULTS_DIR, 'stage1_cnnlstm_results.json')
        with open(results_path, 'w') as f:
            json.dump(results_summary, f, indent=2)
        mlflow.log_artifact(results_path)

        logger.info(f"Best Val Accuracy: {best_val_acc:.4f}")
        logger.info("\nSAVED FILES:")
        logger.info(f"   Best model: {config.MODEL_DIR}/best_cnnlstm_stage1.pth")
        logger.info(f"   Results summary: {results_path}")
        logger.info(f"   Visualizations: {config.RESULTS_DIR}/")
        logger.info(f"   MLFlow tracking: {config.MLFLOW_TRACKING_URI}")

        return best_model, history, results_summary

    except Exception as e:
        logger.error(f"Training pipeline failed: {e}")
        import traceback; traceback.print_exc()
        return None, None, None

In [70]:
def inspect_dataset_root(data_dir: str, max_corrupt_check_per_seq: int = 3):
    """
    Struktur asumsi:
      data_dir/
        0/
          seq_001/ frame_0001.jpg ...
          seq_002/ ...
        1/
          seq_101/ ...
    Output:
      - sequences: list[dict] = [{"sequence_id", "label", "frame_paths"}]
      - prints: ringkasan super detail soal isi dataset SEBELUM proses apa pun
    """
    root = Path(data_dir)
    if not root.exists():
        raise FileNotFoundError(f"DATA_DIR nggak ada: {root}")

    sequences = []
    empty_sequences = []
    non_image_files = []
    exts_ok = {".jpg", ".jpeg", ".png", ".bmp"}

    # 1) Scan folder per label
    for label in [0, 1]:
        class_dir = root / str(label)
        if not class_dir.exists():
            print(f"[WARN] Folder kelas tidak ditemukan: {class_dir}")
            continue
        seq_dirs = sorted([p for p in class_dir.iterdir() if p.is_dir()])

        for sd in seq_dirs:
            # kumpulkan file gambar valid (by extension)
            all_files = sorted([p for p in sd.iterdir() if p.is_file()])
            imgs = [p for p in all_files if p.suffix.lower() in exts_ok]
            others = [p for p in all_files if p.suffix.lower() not in exts_ok]
            non_image_files.extend(others)

            if len(imgs) == 0:
                empty_sequences.append(sd)
                continue

            sequences.append({
                "sequence_id": f"{label}_{sd.name}",
                "label": label,
                "frame_paths": [str(p) for p in imgs],  # simpan string path
            })

    # 2) Ringkasan dasar
    n_seq = len(sequences)
    by_label = Counter([s["label"] for s in sequences])

    print("\n================ RAW DATA SUMMARY ================")
    print(f"Root           : {root.resolve()}")
    print(f"Total sequences: {n_seq}")
    for k in sorted(by_label.keys()):
        print(f"  Label {k}      : {by_label[k]} sequences")

    if empty_sequences:
        print(f"\n[WARN] Empty sequences (tanpa frame): {len(empty_sequences)}")
        for p in empty_sequences[:10]:
            print(f"   - {p}")
        if len(empty_sequences) > 10:
            print(f"   ... ({len(empty_sequences)-10} lagi)")

    if non_image_files:
        print(f"\n[INFO] Non-image files terdeteksi: {len(non_image_files)}")
        for p in non_image_files[:10]:
            print(f"   - {p.name}")
        if len(non_image_files) > 10:
            print(f"   ... ({len(non_image_files)-10} lagi)")

    # 3) Statistik panjang sequence (jumlah frame)
    lens = np.array([len(s["frame_paths"]) for s in sequences], dtype=int)
    if len(lens) > 0:
        q = np.quantile(lens, [0, 0.25, 0.5, 0.75, 1.0])
        print("\nSequence length stats (jumlah frame per sequence):")
        print(f"  Min/25%/Med/75%/Max : {int(q[0])} / {int(q[1])} / {int(q[2])} / {int(q[3])} / {int(q[4])}")
        print(f"  Mean ± Std          : {lens.mean():.2f} ± {lens.std():.2f}")

        # histogram sederhana
        hist = defaultdict(int)
        for L in lens:
            # bucketin per 5 frame biar kebaca
            bucket = int(L // 5) * 5
            hist[bucket] += 1
        top = sorted(hist.items())[:10]
        print("  Contoh histogram (bucket 5‑frame):")
        for b, c in top:
            print(f"    {b:>3d}–{b+4:>3d} : {c}")

    # 4) Cek cepat korup/ga kebaca (sample beberapa frame per sequence)
    #    Tujuan: sebelum training, kita tahu ada file rusak atau tidak.
    bad_images = []
    for s in sequences[: min(len(sequences), 50)]:  # batasi 50 seq biar cepat
        for p in s["frame_paths"][:max_corrupt_check_per_seq]:
            try:
                with Image.open(p) as im:
                    im.verify()  # cepat, cek header
            except Exception as e:
                bad_images.append((p, repr(e)))
    if bad_images:
        print(f"\n[ERROR] Ditemukan file gambar korup/tak bisa dibaca: {len(bad_images)}")
        for p, err in bad_images[:10]:
            print(f"   - {p} :: {err}")
        if len(bad_images) > 10:
            print(f"   ... ({len(bad_images)-10} lagi)")
    else:
        print("\n[OK] Sampel file gambar valid (cek cepat)")

    # 5) Tampilkan contoh 1–2 sequence (path frame awal/akhir)
    print("\nContoh sequence (preview path):")
    for s in sequences[:2]:
        fps = s["frame_paths"]
        print(f"  {s['sequence_id']} | label={s['label']} | frames={len(fps)}")
        print(f"    first 3 : {fps[:3]}")
        print(f"    last  3 : {fps[-3:]}")

    print("==================================================\n")
    return sequences

# === PANGGIL DI SINI ===
DATA_DIR = "the_dataset"  # ganti sesuai folder dataset kamu
sequences = inspect_dataset_root(DATA_DIR)


================ RAW DATA SUMMARY ================
Root           : C:\Users\andre\Documents\Binus\Lomba\compfest\IllegalParkingDetection-Compfest2025\the_dataset
Total sequences: 18
  Label 0      : 5 sequences
  Label 1      : 13 sequences

Sequence length stats (jumlah frame per sequence):
  Min/25%/Med/75%/Max : 16 / 16 / 16 / 16 / 26
  Mean ± Std          : 18.28 ± 4.13
  Contoh histogram (bucket 5‑frame):
     15– 19 : 14
     25– 29 : 4

[OK] Sampel file gambar valid (cek cepat)

Contoh sequence (preview path):
  0_event_v020_p028_180 | label=0 | frames=16
    first 3 : ['the_dataset\\0\\event_v020_p028_180\\frame_000.jpg', 'the_dataset\\0\\event_v020_p028_180\\frame_001.jpg', 'the_dataset\\0\\event_v020_p028_180\\frame_002.jpg']
    last  3 : ['the_dataset\\0\\event_v020_p028_180\\frame_013.jpg', 'the_dataset\\0\\event_v020_p028_180\\frame_014.jpg', 'the_dataset\\0\\event_v020_p028_180\\frame_015.jpg']
  0_event_v2247_p2249_24186 | label=0 | frames=16
    first 3 : ['the_datas

In [63]:
def _build_val_transform(config):
    from torchvision import transforms
    return transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop(config.IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
    ])

def _load_sequence_frames(seq_dir, transform, target_len):
    # kumpulkan file gambar, sort biar urut waktu
    exts = ('*.jpg','*.jpeg','*.png','*.bmp')
    paths = []
    for e in exts:
        paths.extend(glob.glob(os.path.join(seq_dir, e)))
    paths = sorted(paths)
    if len(paths) == 0:
        raise FileNotFoundError(f"Tidak ada frame di folder: {seq_dir}")

    # sampling/padding ke target_len
    if len(paths) >= target_len:
        idx = torch.linspace(0, len(paths)-1, steps=target_len).round().long().tolist()
        sel = [paths[i] for i in idx]
        length = target_len
    else:
        sel = paths[:]
        # pad dengan frame terakhir
        sel += [paths[-1]] * (target_len - len(paths))
        length = len(paths)

    # load & transform -> (T,C,H,W)
    frames = []
    for p in sel:
        img = Image.open(p).convert('RGB')
        frames.append(transform(img))
    x_tchw = torch.stack(frames, dim=0)  # (T,C,H,W)
    return x_tchw, length, sel

def run_prediction_test(seq_dir: str):
    # gunakan config untuk CNN+LSTM (bukan CNNConfig lama)
    config = CNNLSTMConfig()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # siapkan model (harus sama dengan yang dipakai saat training)
    model = CNNLSTMModel(
        num_classes=config.NUM_CLASSES,
        cnn_feature_size=config.CNN_FEATURE_SIZE,
        lstm_hidden=config.LSTM_HIDDEN_SIZE,
        lstm_layers=config.LSTM_NUM_LAYERS,
        bidirectional=False,
        dropout=config.DROPOUT,
        freeze_backbone=True,  # stage-1
    ).to(device)
    model.eval()

    # load checkpoint stage-1
    ckpt_path = os.path.join(config.MODEL_DIR, 'best_cnnlstm_stage1.pth')
    if not os.path.exists(ckpt_path):
        raise FileNotFoundError(f"Checkpoint tidak ditemukan: {ckpt_path}")
    ckpt = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(ckpt['model_state_dict'])

    # siapkan sequence tensor
    val_tf = _build_val_transform(config)
    x_tchw, length, used_paths = _load_sequence_frames(seq_dir, val_tf, config.SEQ_LEN)

    # ke batch: (1,T,C,H,W), lengths: (1,)
    x_btchw = x_tchw.unsqueeze(0).to(device)
    lengths = torch.tensor([length], dtype=torch.long, device=device)

    with torch.no_grad():
        logits = model(x_btchw, lengths)        # (1,num_classes)
        probs  = F.softmax(logits, dim=1)[0]    # (num_classes,)
        pred   = torch.argmax(probs).item()

    class_map = {0: 'normal_parking', 1: 'person_exiting'}
    result = {
        'prediction': pred,
        'class_name': class_map.get(pred, str(pred)),
        'probabilities': {
            'normal_parking': float(probs[0].item()),
            'person_exiting': float(probs[1].item())
        },
        'seq_dir': seq_dir,
        'used_frames': used_paths,
        'effective_length': int(length),
        'target_len': int(config.SEQ_LEN),
    }

    print(f"Prediction: {result['class_name']}")
    print(f"  normal_parking  : {result['probabilities']['normal_parking']:.4f}")
    print(f"  person_exiting  : {result['probabilities']['person_exiting']:.4f}")
    print(f"Frames used: {result['effective_length']}/{result['target_len']} • {seq_dir}")
    return result

In [68]:
if __name__ == "__main__":
    best_model, history, results = main()

    if best_model is not None:
        # ganti path ini dengan folder sequence yang mau kamu tes
        seq_dir = "the_dataset/1/seq_101"   # <-- contoh
        try:
            _ = run_prediction_test(seq_dir=seq_dir)
        except Exception as e:
            logger.error(f"Prediction test failed: {e}")
    else:
        logger.error("\nStage 1 CNN+LSTM implementation failed")


Epoch 1/10:   0%|          | 0/7 [00:00<?, ?it/s]

KeyboardInterrupt: 